---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
# university_towns.txt

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # https://programminghistorian.org/lessons/working-with-text-files#reading-from-a-text-file
    # https://stackoverflow.com/questions/17262256/how-to-read-one-single-line-of-csv-data-in-python
    # https://stackoverflow.com/questions/20894525/how-to-remove-parentheses-and-all-data-within-using-pandas-python
    # https://stackoverflow.com/questions/3437059/does-python-have-a-string-contains-substring-method
    # regex
    # for pandas: https://stackoverflow.com/questions/20894525/how-to-remove-parentheses-and-all-data-within-using-pandas-python
    # for re: https://stackoverflow.com/questions/5658369/how-to-input-a-regex-in-string-replace
    # https://stackoverflow.com/questions/904746/how-to-remove-all-characters-after-a-specific-character-in-python
    # https://docs.python.org/3/library/re.html
    # www.regular-expressions.info.
    
    university_df = pd.DataFrame()
    currentState = ''
    currentCity = ''
    
    import csv
    
    with open('university_towns.txt', newline='\n') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
#             rowString = str(row)
            rowString = ''.join(row)
#             print(type(rowString))
            if '[edit]' in rowString:
#                 line = re.sub(r"</?\[\d+>", "", line)
#                 currentState = re.sub(r"\[edit\]","",rowString).strip()
                currentState = rowString.split("[")[0]
                continue
#                 print(currentState)
#             elif '(' in rowString:     
            else:
#                 currentCity = re.sub(r"\(.*?\)","",rowString).strip()
#                 currentCity = re.sub(r"\[.*?\]","",currentCity).strip()
#                 currentCity = re.sub(r"\s\s\s","\s",currentCity).strip()
#                 currentCity = re.sub(r"\s\s","\s",currentCity).strip()
                currentCity = rowString.split('(')[0]
#                 print(currentCity)
#             else:
#                 print(rowString)
#                 continue
                
            # add to pandas dataframe
            currentSeries = pd.Series({'State': currentState,
                                     'RegionName': currentCity})
            university_df = university_df.append(currentSeries, ignore_index=True)
            
            #https://stackoverflow.com/questions/34446767/stripping-all-trailing-empty-spaces-in-a-column-of-a-pandas-dataframe
            university_df['State'] = university_df['State'].str.strip()
            university_df['RegionName'] = university_df['RegionName'].str.strip()
    
    return university_df

df = get_list_of_university_towns()
# print(len(df['State'].unique()))
df

# === exception case - ignored issues ===
# The Colleges of Worcester Consortium:
# The Five College Region of Western Massachusetts:
# Faribault, South Central College
# North Mankato, South Central College

# === unused code ===
# line = re.sub(r"""
#   (?x) # Use free-spacing mode.
#   <    # Match a literal '<'
#   /?   # Optionally match a '/'
#   \[   # Match a literal '['
#   \d+  # Match one or more digits
#   >    # Match a literal '>'
#   """, "", line)

#     import csv
#     with open('some.csv', newline='') as f:
#       reader = csv.reader(f)
#       for row in reader:
#         print(row)

,RegionName,State
0,Auburn,Alabama
1,Florence,Alabama
2,Jacksonville,Alabama
3,Livingston,Alabama
4,Montevallo,Alabama
5,Troy,Alabama
6,Tuscaloosa,Alabama
7,Tuskegee,Alabama
8,Fairbanks,Alaska
9,Flagstaff,Arizona


In [7]:
# From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the United States in current dollars 
# (use the chained value in 2009 dollars), in quarterly intervals, 
# in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.

# A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
# A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
# A recession bottom is the quarter within a recession which had the lowest GDP.
# A university town is a city which has a high percentage of university students compared to the total population of the city.

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    # recession start time had 2 consecutive quarters of GDP decline
    
    recessionStart = ""
    # gdplev.xls
    ## Read recession data
    gdp_df = pd.read_excel('gdplev.xls',header=None,skiprows=220,parse_cols=[4,6], names=['quarter', 'gdp'])
    gdp_0_quarter = ''
    gdp_1 = 0
    gdp_1_quarter = ''
    gdp_2 = 0
    gdp_2_quarter = ''
    minIndex = 0
    maxIndex = np.max(list(gdp_df.index))
    for i in range(minIndex, maxIndex):
        row = gdp_df.iloc[i]
        if((row['gdp']<gdp_2) and (gdp_2<gdp_1)):
            # print(row['quarter'])
            recessionStart = gdp_2_quarter#(row['quarter'])
            break # first one is all we need
        gdp_0_quarter = gdp_1_quarter
        gdp_1 = gdp_2
        gdp_1_quarter = gdp_2_quarter
        gdp_2 = row['gdp']        
        gdp_2_quarter = row['quarter']
        
    return recessionStart


get_recession_start()

'2008q1'

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    recessionEnd = ""
    # gdplev.xls
    ## Read recession data
    gdp_df = pd.read_excel('gdplev.xls',header=None,skiprows=220,parse_cols=[4,6], names=['quarter', 'gdp'])
    gdp_1 = 1000000
    gdp_2 = 1000000
    recessionStart = get_recession_start()
    minIndex = gdp_df[gdp_df['quarter']==recessionStart].index[0] # use recession start
    maxIndex = np.max(list(gdp_df.index))
    for i in range(minIndex, maxIndex):
        row = gdp_df.iloc[i]
        if((row['gdp']>gdp_2) and (gdp_2>gdp_1)):
            # print(row['quarter'])
            recessionEnd = (row['quarter'])
            break # first one is all we need
        gdp_1 = gdp_2
        gdp_2 = row['gdp']
        
    return recessionEnd


get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recessionBottom = ""
    # gdplev.xls
    ## Read recession data
    gdp_df = pd.read_excel('gdplev.xls',header=None,skiprows=220,parse_cols=[4,6], names=['quarter', 'gdp'])
    min_gdp = 1000000
    recessionStart = get_recession_start()
    recessionEnd = get_recession_end()
    minIndex = gdp_df[gdp_df['quarter']==recessionStart].index[0] # use recession start
    maxIndex = gdp_df[gdp_df['quarter']==recessionEnd].index[0] # use recession start
    for i in range(minIndex, maxIndex):
        row = gdp_df.iloc[i]
        if((row['gdp']<min_gdp)):
            # print(row['quarter'])
            recessionBottom = (row['quarter'])
            min_gdp = row['gdp']
            # break # first one is all we need
        
    return recessionBottom

get_recession_bottom()

'2009q2'

In [24]:
import numpy as np


def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # City_Zhvi_AllHomes.csv

    ## define kept columns
    ### ignored columns
    # 'RegionID',
    # 'Metro', 'CountyName', 'SizeRank',
    # '1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09',
    # '1996-10', '1996-11', '1996-12', '1997-01', '1997-02', '1997-03',
    # '1997-04', '1997-05', '1997-06', '1997-07', '1997-08', '1997-09',
    # '1997-10', '1997-11', '1997-12', '1998-01', '1998-02', '1998-03',
    # '1998-04', '1998-05', '1998-06', '1998-07', '1998-08', '1998-09',
    # '1998-10', '1998-11', '1998-12', '1999-01', '1999-02', '1999-03',
    # '1999-04', '1999-05', '1999-06', '1999-07', '1999-08', '1999-09',
    # '1999-10', '1999-11', '1999-12',
    columnNames = [
        'RegionName', 'State', '2000-01', '2000-02', '2000-03', '2000-04',
        '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10',
        '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04',
        '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10',
        '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04',
        '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10',
        '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04',
        '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10',
        '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04',
        '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10',
        '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04',
        '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-10',
        '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04',
        '2006-05', '2006-06', '2006-07', '2006-08', '2006-09', '2006-10',
        '2006-11', '2006-12', '2007-01', '2007-02', '2007-03', '2007-04',
        '2007-05', '2007-06', '2007-07', '2007-08', '2007-09', '2007-10',
        '2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04',
        '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10',
        '2008-11', '2008-12', '2009-01', '2009-02', '2009-03', '2009-04',
        '2009-05', '2009-06', '2009-07', '2009-08', '2009-09', '2009-10',
        '2009-11', '2009-12', '2010-01', '2010-02', '2010-03', '2010-04',
        '2010-05', '2010-06', '2010-07', '2010-08', '2010-09', '2010-10',
        '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04',
        '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10',
        '2011-11', '2011-12', '2012-01', '2012-02', '2012-03', '2012-04',
        '2012-05', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10',
        '2012-11', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04',
        '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10',
        '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04',
        '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10',
        '2014-11', '2014-12', '2015-01', '2015-02', '2015-03', '2015-04',
        '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10',
        '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
        '2016-05', '2016-06', '2016-07', '2016-08'
    ]
    removeCols = [
        '2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06',
        '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12',
        '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06',
        '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12',
        '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06',
        '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12',
        '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06',
        '2003-07', '2003-08', '2003-09', '2003-10', '2003-11', '2003-12',
        '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06',
        '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12',
        '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06',
        '2005-07', '2005-08', '2005-09', '2005-10', '2005-11', '2005-12',
        '2006-01', '2006-02', '2006-03', '2006-04', '2006-05', '2006-06',
        '2006-07', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12',
        '2007-01', '2007-02', '2007-03', '2007-04', '2007-05', '2007-06',
        '2007-07', '2007-08', '2007-09', '2007-10', '2007-11', '2007-12',
        '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06',
        '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12',
        '2009-01', '2009-02', '2009-03', '2009-04', '2009-05', '2009-06',
        '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12',
        '2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
        '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
        '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
        '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
        '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
        '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
        '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
        '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
        '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
        '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
        '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
        '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
        '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
        '2016-07', '2016-08'
    ]

    # City_Zhvi_AllHomes.csv
    ## Read recession data
    housing_df = pd.read_csv(
        'City_Zhvi_AllHomes.csv', usecols=columnNames,
        index_col=[0, 1])  #,parse_cols=[4,7],index_col=[4,6])

    allCols = housing_df.columns.values.tolist()
    for aCol in allCols:
        yr = aCol.split('-')[0]
        mo = aCol.split('-')[1]
        if (mo == "01"):
            newCol = yr + 'q1'
            col1 = aCol
            col2 = yr + "-" + "02"
            col3 = yr + "-" + "03"
            temp_df = pd.DataFrame(data=housing_df, columns=[col1, col2, col3])
            housing_df[newCol] = temp_df.mean(axis=1)
        elif (mo == "04"):
            newCol = yr + 'q2'
            col1 = aCol
            col2 = yr + "-" + "05"
            col3 = yr + "-" + "06"
            temp_df = pd.DataFrame(data=housing_df, columns=[col1, col2, col3])
            housing_df[newCol] = temp_df.mean(axis=1)
        elif (mo == "07"):
            newCol = yr + 'q3'
            col1 = aCol
            col2 = yr + "-" + "08"
            col3 = yr + "-" + "09"
            temp_df = pd.DataFrame(data=housing_df, columns=[col1, col2, col3])
            housing_df[newCol] = temp_df.mean(axis=1)
        elif (mo == "10"):
            newCol = yr + 'q4'
            col1 = aCol
            col2 = yr + "-" + "11"
            col3 = yr + "-" + "12"
            temp_df = pd.DataFrame(data=housing_df, columns=[col1, col2, col3])
            housing_df[newCol] = temp_df.mean(axis=1)
    housing_df = housing_df.drop(removeCols, 1)

    housing_df = housing_df.reset_index()
    housing_df["State"] = housing_df["State"].apply(
        lambda x: states[x])  #, axis=1)
    housing_df = housing_df.set_index(['State', 'RegionName'])

    return housing_df


# housing_df = pd.read_csv(
#         'City_Zhvi_AllHomes.csv')
housing_df = convert_housing_data_to_quarters()
# # housing_df["State"] = housing_df["State"].apply(lambda x: states[x])
# convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]
housing_df

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [26]:
# housing_df.loc["Texas"].loc["Austin"].loc["2010q3"] 
# housing_df.index

# university_df = get_list_of_university_towns()
# university_df

202266.66666666666

In [9]:
# # matching Region Name approximately is not good enough

# def isSchoolTownData_relax(housing_df, university_df):
#     housing_df = housing_df.reset_index()
#     housing_df["State"] = housing_df["State"].apply(lambda x: states[x])  #, axis=1)
    
#     isState = housing_df["State"].isin(university_df["State"])
#     isRegion = housing_df["RegionName"].isin(university_df["RegionName"])
#     #['Berkeley']
    
#     isUniTwn = isRegion&isState
    
#     return isUniTwn #housing_df[isUniTwn] #isUniTwn[isUniTwn==True]

# housing_df[isSchoolTownData_relax(housing_df, university_df)]

In [34]:
def isSchoolTownData(housing_df, university_df):
    """determine if the the housing data row belongs to a university town"""

    housing_df = housing_df.reset_index()
    #     housing_df = housing_df.set_index(['RegionName', 'State'])
    # a possibly too inefficient way to get a list of school by iterate through university_df
    # use "states" object for matching state
    isUniTwn = ((housing_df["RegionName"] == university_df["RegionName"][0]))
    #     isUniTwn = ((housing_df["RegionName"] == university_df["RegionName"][0]) &
    #                 (housing_df["State"] == university_df["State"][0]))

    #     (((housing_df["RegionName"] == university_df["RegionName"][0]) |
    #                  (housing_df["Metro"] == university_df["RegionName"][0]) |
    #                  (housing_df["CountyName"] == university_df["RegionName"][0]))
    #                 & (housing_df["State"] == university_df["State"][0]))

    #     ((housing_df["RegionName"] == university_df["RegionName"][0]) &
    #                 (housing_df["State"] == university_df["State"][0]))

    #     (((housing_df["RegionName"] == university_df["RegionName"][0]) |
    #                  (housing_df["CountyName"] == university_df["RegionName"][0]))
    #                 & (housing_df["State"] == university_df["State"][0]))

    for index, row in university_df.iterrows():
        curr_isUniTwn = ((housing_df["RegionName"] == row["RegionName"]))
        #         curr_isUniTwn = ((housing_df["RegionName"] == row["RegionName"]) &
        #                          (housing_df["State"] == row["State"]))

        #         (((housing_df["RegionName"] == row["RegionName"])
        #                           | (housing_df["Metro"] == row["RegionName"])
        #                           | (housing_df["CountyName"] == row["RegionName"])) &
        #                          (housing_df["State"] == row["State"]))

        #         ((housing_df["RegionName"] == row["RegionName"]) &
        #                 (housing_df["State"] == row["State"]))

        #         (((housing_df["RegionName"] == row["RegionName"])
        #             | (housing_df["CountyName"] == row["RegionName"])) &
        #                          (housing_df["State"] == row["State"]))

        isUniTwn = (isUniTwn | curr_isUniTwn)

    uniTwn_df = housing_df[isUniTwn]
    return uniTwn_df


def isNotSchoolTownData(housing_df, university_df):
    """determine if the the housing data row belongs to a university town"""

    housing_df = housing_df.reset_index()
    #     housing_df["State"] = housing_df["State"].apply(
    #         lambda x: states[x])  #, axis=1)
    # a possibly too inefficient way to get a list of school by iterate through university_df
    # use "states" object for matching state
    isUniTwn = ((housing_df["RegionName"] == university_df["RegionName"][0]))
    #     isUniTwn = ((housing_df["RegionName"] == university_df["RegionName"][0]) &
    #                 (housing_df["State"] == university_df["State"][0]))

    #     (((housing_df["RegionName"] == university_df["RegionName"][0]) |
    #                  (housing_df["Metro"] == university_df["RegionName"][0]) |
    #                  (housing_df["CountyName"] == university_df["RegionName"][0]))
    #                 & (housing_df["State"] == university_df["State"][0]))

    #     ((housing_df["RegionName"] == university_df["RegionName"][0]) &
    #                 (housing_df["State"] == university_df["State"][0]))

    #     (((housing_df["RegionName"] == university_df["RegionName"][0]) |
    #                  (housing_df["CountyName"] == university_df["RegionName"][0]))
    #                 & (housing_df["State"] == university_df["State"][0]))

    for index, row in university_df.iterrows():
        curr_isUniTwn = ((housing_df["RegionName"] == row["RegionName"]))
        #         curr_isUniTwn = ((housing_df["RegionName"] == row["RegionName"]) &
        #                  (housing_df["State"] == row["State"]))

        #         (((housing_df["RegionName"] == row["RegionName"])
        #                           | (housing_df["Metro"] == row["RegionName"])
        #                           | (housing_df["CountyName"] == row["RegionName"])) &
        #                          (housing_df["State"] == row["State"]))

        #         ((housing_df["RegionName"] == row["RegionName"]) &
        #                 (housing_df["State"] == row["State"]))

        #         (((housing_df["RegionName"] == row["RegionName"])
        #             | (housing_df["CountyName"] == row["RegionName"])) &
        #                          (housing_df["State"] == row["State"]))

        isUniTwn = (isUniTwn | curr_isUniTwn)

    uniTwn_df = housing_df[~isUniTwn]
    return uniTwn_df

In [48]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    # Hypothesis: University towns have their mean housing prices less effected by recessions.
    # Run a t-test to compare the ratio of the mean price of houses in university towns the
    # quarter before the recession starts compared to the recession bottom.
    # (price_ratio=quarter_before_recession/recession_bottom)

    # data between recession start and recession bottom
    df = convert_housing_data_to_quarters()
    start_index = list(df.columns).index(get_recession_start())
    end_index = list(df.columns).index(get_recession_bottom())
    df = df.iloc[:, [start_index, end_index]]#[start_index - 1, end_index + 1]]
    df["priceRatio"] = df.iloc[:, 0] / df.iloc[:, 1]

    university_df = get_list_of_university_towns()

    # data separted by university town and non-university town
    uniTwn_df = isSchoolTownData(df, university_df)
    regTwn_df = isNotSchoolTownData(df, university_df)

    # check if two grousp are the same with t-test
    from scipy import stats
    #     stats.ttest_ind?
    ttest = stats.ttest_ind(regTwn_df['priceRatio'].dropna(),
                            uniTwn_df['priceRatio'].dropna())
    # ttest.pvalue
    # ttest.statistic
    # ttest
    different = False
    p_val = ttest.pvalue
    better = False
    if p_val < 0.5:
        different = True
    if regTwn_df['priceRatio'].mean() < uniTwn_df['priceRatio'].mean():
        better = "non-university town"
    else:
        better = 'university town'#True
    
    return (different, p_val, better)


#regTwn_df #uniTwn_df

print(get_recession_start())
print(get_recession_bottom())
run_ttest()

2008q1
2009q2


(True, 3.9204110809200024e-05, 'university town')

In [49]:

# '''First creates new data showing the decline or growth of housing prices
# between the recession start and the recession bottom. Then runs a ttest
# comparing the university town values to the non-university towns values, 
# return whether the alternative hypothesis (that the two groups are the same)
# is true or not as well as the p-value of the confidence. 

# Return the tuple (different, p, better) where different=True if the t-test is
# True at a p<0.01 (we reject the null hypothesis), or different=False if 
# otherwise (we cannot reject the null hypothesis). The variable p should
# be equal to the exact p value returned from scipy.stats.ttest_ind(). The
# value for better should be either "university town" or "non-university town"
# depending on which has a lower mean price ratio (which is equivilent to a
# reduced market loss).'''
    
# # Hypothesis: University towns have their mean housing prices less effected by recessions.
# # Run a t-test to compare the ratio of the mean price of houses in university towns the
# # quarter before the recession starts compared to the recession bottom.
# # (price_ratio=quarter_before_recession/recession_bottom)

# # data between recession start and recession bottom
# df = convert_housing_data_to_quarters()
# start_index = list(df.columns).index(get_recession_start())
# end_index = list(df.columns).index(get_recession_bottom())
# df = df.iloc[:, [start_index, end_index]]
# df["priceRatio"] = df.iloc[:, 0] / df.iloc[:, 1]
# # df
# university_df = get_list_of_university_towns()
# # university_df
# # # data separted by university town and non-university town
# uniTwn_df = isSchoolTownData(df, university_df)
# uniTwn_df


In [42]:
# regTwn_df = isNotSchoolTownData(df, university_df)
# regTwn_df
# # # check if two grousp are the same with t-test
# # from scipy import stats
# # #     stats.ttest_ind?
# # ttest = stats.ttest_ind(regTwn_df['priceRatio'].dropna(),
# #                         uniTwn_df['priceRatio'].dropna())
# # # ttest.pvalue
# # # ttest.statistic
# # # ttest
# # different = False
# # p_val = ttest.pvalue
# # better = False
# # if p_val < 0.5:
# #     different = True
# # if regTwn_df['priceRatio'].mean() > uniTwn_df['priceRatio'].mean():
# #     better = "non-university town"
# # else:
# #     better = 'university town'#True

In [12]:
### BASIC MATH STUFF ######################

# # aList = [1,2,2,4,5,6,7,9,12]
# # aList = [1,2,2,3,3,4,12,30,90]
# aList = [3,3,90,100]

# su = np.sum(aList)
# mean = np.mean(aList)
# median = np.median(aList)

# print("sum is: " + str(su))
# print("mean is: " + str(mean))
# print("median is: " + str(median))

# from statistics import mode

# mod = mode(aList)
# print("mode is: " + str(mod))

In [13]:
### METHOD 1: iterate over stack rows - too slow ######################
# see method 2 in the convert_housing_data_to_quarters function

# import numpy as np

# housing_df = housing_df.stack()
# housing_df = housing_df.reset_index()
# housing_df = housing_df.set_index(['RegionName', 'State'])
# ## rename columns
# housing_df.columns = ['month', 'medPrice']

# # type(df['month'][0])
# final_cols = ['RegionName', 'State', 'quarter', 'avgPrice']
# # final_indx = 
# final_housing_df = pd.DataFrame(columns=final_cols, index = None)
# mo1_index = None
# mo2_index = None
# mo1_val = None
# mo2_val = None
# i=0
# for index, row in df.iterrows():
#     # print([index, row['month'], row['medPrice']])
#     year = row['month'].split('-')[0]
#     mo = row['month'].split('-')[1]
#     val = row['medPrice']
#     # print(index,mo1_index,mo2_index)
#     if((index==mo2_index and mo2_index==mo1_index)):
#         if(mo=='03'):
#             colLabel = (year+str('q1'))
#             avgVal = np.mean([mo1_val, mo2_val, val])
#             final_housing_df = final_housing_df.append(pd.Series({'RegionName': index[0], 'State': index[1], 'quarter':colLabel,'avgPrice':avgVal}), ignore_index=True)
#             break
#     else:
#         print(index==mo2_index and mo2_index==mo1_index)
#         mo1_index = mo2_index
#         mo2_index = index
#         mo1_val = mo2_val
#         mo2_val = val


### DEBUG NOTES ######################
# https://pandas.pydata.org/pandas-docs/stable/indexing.html
# df.loc[row_indexer,column_indexer]

# df.iloc[0,:]

# %timeit df.iterrows()
# 310 ns ± 7.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
# %timeit df.itertuples()
# 592 µs ± 55.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# %timeit df.iteritems()
# 307 ns ± 4.09 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

# https://github.com/pandas-dev/pandas/issues/10334
# In [10]: x = pd.DataFrame({'x': range(10000)})
# In [11]: %timeit list(x.iterrows())
# 1 loops, best of 3: 383 ms per loop
# In [12]: %timeit list(x.itertuples())
# 1000 loops, best of 3: 1.39 ms per loop

# https://stackoverflow.com/questions/25358511/improve-iteritems-performance-or-a-better-way-to-loop-through-python-dictionary
# if (m, l, k) in xDict:
#     om, ol, ok = xDict[m, l, k]

# https://www.tutorialspoint.com/python_pandas/python_pandas_iteration.htm
# for key,value in df.iteritems():
#     print key,value
# for row_index,row in df.iterrows():
#     print row_index,row
# for row in df.itertuples():
#     print row